In [1]:
import os, sys
sys.path.append("..")

In [3]:
from src.pauli import PauliElement

In [5]:
n = 10
p0 = PauliElement(nx=3, nz= 6, n=n, weight=0.124, f=0, set_phase=False)
p1 = PauliElement(nx=5, nz= 5, n=n, weight=0.124, f=0, set_phase=False)

In [6]:
%%timeit
p2 = p0@p1

3.05 μs ± 56.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [7]:
p2 = p0@p1

In [13]:
PauliElement.__repr__(p2)

'Pauli(xz=(6, 3), IIIIIIIXYZ, n=10, weight=0.015375999999999999)'

In [9]:
p00_.pstr

NameError: name 'p00_' is not defined

In [7]:
p00_.sym_code

(3, 6)

In [8]:
p00_.ij_code

(6, 5)

## Speed comparsion

In [11]:
from numbers import Number
from functools import reduce

import numpy as np

def p__str__(self):
        return f"{self.pstr}"
def p__repr__(self):
        return f"Pauli(xz=({self.nx}, {self.nz}), {self.pstr}, n={self.n}, weight={self.weight})"
def p_to_pennylane(self, words=False):
        from pennylane.pauli import string_to_pauli_word
        from pennylane import I, PauliX, PauliY, PauliZ
        if words:
            map_dict = {i:i for i in (range(self.n))}
            terms = string_to_pauli_word(self.string, wire_map=map_dict)
        else:
            ops = {
                "I": lambda i: I(i),
                "X": lambda i: PauliX(i),
                "Y": lambda i: PauliY(i),
                "Z": lambda i: PauliZ(i),
                   }
            opers = [ops[s](i) for i, s in enumerate(reversed(self.string))]
            terms =reduce(lambda x, y: x@y, opers)
        return self.coef, terms
    
def p_to_qiskit(self):
        from qiskit.quantum_info import Pauli as qiskit_Pauli
        x_array = np.flip(np.array([int(b) for b in format(self.nx, f"0{self.n}b")], dtype=np.uint))
        z_array = np.flip(np.array([int(b) for b in format(self.nz, f"0{self.n}b")], dtype=np.uint))
        return self.weight, qiskit_Pauli((z_array, x_array))

# Basic Pauli matrices
I = np.eye(2, dtype=int)
X = np.matrix([[0, 1], [1, 0]], dtype=int)
Y = np.matrix([[0, 1], [-1, 0]], dtype=int) # omit -1j* phase
Z = np.matrix([[1, 0],[0, -1]], dtype = int)

# Pauli matrix by name
PAULI_MATRICES = {
    "I": I,
    "X": X,
    "Y": Y,
    "Z": Z
}
def krons(*oper_list)->np.matrix:
    """Kronecker product(=Tensor product of matrix).
    
    Returns:
        np.matrix: Kronecker producted matrix of the given oredred matrix.
    """
    if len(oper_list) == 1:
        oper_list = oper_list[0]
    return reduce(np.kron, oper_list)

def pstr2mat(pstr:str)->np.matrix:
        result = []
        for p in pstr:
            result.append(PAULI_MATRICES[p])
        return krons(result)
def p_to_matrix(self):
      return self.weight * (-1j)**self.f * pstr2mat(self.pstr)
       

_PauliElement.__str__ = p__str__
_PauliElement.__repr__ = p__repr__
_PauliElement.to_matrix = p_to_matrix
_PauliElement.to_pennylane = p_to_pennylane
_PauliElement.to_qiskit = p_to_qiskit


In [17]:
n = 1000
p00 = _PauliElement(nx=0, nz= 1, n=n, weight=0.124, f=0, set_phase=False)
p01 = _PauliElement(nx=5, nz= 5, n=n, weight=0.124, f=0, set_phase=False)
w00, q_p00 = p00.to_qiskit()
w01, q_p01 = p01.to_qiskit()


In [18]:
p00.to_matrix()

KeyboardInterrupt: 

In [19]:
%%timeit
p02 = p00@p01

3.35 μs ± 399 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Compare with Qiskit

In [37]:
n = 1024
p00 = _PauliElement(nx=3, nz= 6, n=n, weight=0.124, f=0, set_phase=False)
p01 = _PauliElement(nx=5, nz= 5, n=n, weight=0.124, f=0, set_phase=False)
w00, q_p00 = p00.to_qiskit()
w01, q_p01 = p01.to_qiskit()


In [38]:
q_p00.compose(q_p01)

Pauli('IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII...')

In [39]:
p02= p00@p01
p02.pstr

'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXYZ'

## Speed

### Qiskit

In [40]:
%%timeit
q_p02 = q_p00.compose(q_p01)

6.91 μs ± 176 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### OptTrot

In [41]:
%%timeit
p02 = p00@p01

3.61 μs ± 684 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Numpy binary

In [42]:
import numpy as np

nx1= np.random.randint(0, 2, size=n, dtype=bool)
nz1= np.random.randint(0, 2, size=n, dtype=bool)
nx2= np.random.randint(0, 2, size=n, dtype=bool)
nz2= np.random.randint(0, 2, size=n, dtype=bool)
f1 = 3
f2 = 4


In [43]:
%%timeit
nx3 = np.bitwise_xor(nx1, nx2)
nz3 = np.bitwise_xor(nz1, nz2)
f3 = f1+f2 + 2*np.bitwise_and(nx1, nz2).astype(int).sum() -  np.bitwise_and(nx3, nz3).astype(int).sum()

4.63 μs ± 13.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
